In [1]:
import xml.etree.ElementTree as ET

# Variables
name = "admin"
password = "password"

# Create the root element
root = ET.Element("userRegistry")
root.set("xmlns", "http://www.geoserver.org/security/users")
root.set("version", "1.0")

# Create the users element
users = ET.SubElement(root, "users")

# Create the user element
user = ET.SubElement(users, "user")
user.set("enabled", "true")
user.set("name", name)
user.set("password", password)

# Create the groups element
groups = ET.SubElement(root, "groups")

# Create the XML string
xml_string = ET.tostring(root, encoding="utf-8").decode()

# Print the XML output
print(xml_string)


<userRegistry xmlns="http://www.geoserver.org/security/users" version="1.0"><users><user enabled="true" name="admin" password="password" /></users><groups /></userRegistry>


In [2]:
import utils
import core

config = '../inputs/config.yml'

configuration = utils.read_configuration(config)
print(configuration)

core.create_local_folders_and_files(config)
#users = core.create_user_credentials(config)

#core.xml_users(users)


{'base_path': '../outputs', 'usergroups': [{'name': 'default', 'users': ['admin', 'alice', 'bob']}, {'name': 'new', 'users': ['xavier', 'yael', 'zoe']}], 'paths': {'secret_path': '../outputs/secrets', 'gs_data_dir': '../outputs/gs_data_dir', 'gs_security': '../outputs/gs_data_dir/security', 'gs_sec_usergroup': '../outputs/gs_data_dir/security/usergroup'}, 'stored_secrets': '../outputs/secrets/secrets.txt'}
INFO: Path '../outputs/secrets' already exists.
INFO: Path '../outputs/gs_data_dir' has been created.
INFO: Path '../outputs/gs_data_dir/security' has been created.
INFO: Path '../outputs/gs_data_dir/security/usergroup' has been created.
INFO: secret.txt already exists.
